##### Imports

In [1]:
import asvFormula.bayesianNetworks.bayesianNetwork as bn
from asvFormula.bayesianNetworks import networkSamplesPath
from asvFormula.datasetManipulation import *
from asvFormula.exactASV import showASVandShapleyFor, ASV
from asvFormula.classesSizes.digraph import hasUnderlyingTree
import random 
from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination
import numpy as np

#TODO: I should add classes and assign responsibilities to them. All of the functions are mixed and I have multiple variables flying. 
    # Feature values could be a class.
#TODO: Fix the seed and the random so that I can reproduce the results. Right now it's not changing anything the seed method. 

In [2]:
from importlib import reload 
import asvFormula
import asvFormula.bayesianNetworks.bayesianNetwork as bn

bn = reload(asvFormula.bayesianNetworks.bayesianNetwork)

### Running ASV for Cancer bayesian network

#### Train the decision tree model

In [3]:
cancerNetworkPath = networkSamplesPath + "/cancer.bif"
BNmodel = BIFReader(cancerNetworkPath).get_model()
variableToPredict = "Pollution"
numberOfSamples = 2000
treeMaxDepth = 3

BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeData(BNmodel, variableToPredict, numberOfSamples, treeMaxDepth)

  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.9225


#### Compute the ASV vs the Shapley Value

In [4]:
asvCalc = ASV(BNmodel, dtTreeClassifier, BNInference, valuesPerFeature)

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

#showASVandShapleyFor(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc)


#### Mean prediction of model

In [5]:
# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(cancerNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)

In [6]:
bn.showMeanPredictionOfModel(variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

Mean prediction of model for the variable Pollution
Mean prediction value for the model: 0.9056418232938892
Estimated value for shap explainer: 0.906875
Mean prediction if we only use the bayesian network: 0.9


### Running ASV for Child bayesian network

In [32]:
childNetworkPath = networkSamplesPath + "/child.bif"

def childBNAsTree(childNetworkPath):
    treeBNChild = BIFReader(childNetworkPath).get_model()

    #I remove this edges so that it is a tree and we can work with it
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungParench', 'Grunting')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungParench', 'HypoxiaInO2')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'HypoxiaInO2', 'LowerBodyO2')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'CardiacMixing', 'HypDistrib')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'Sick', 'Age')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungFlow', 'ChestXray')

    assert hasUnderlyingTree(treeBNChild)
    return treeBNChild

treeBNChild = childBNAsTree(childNetworkPath)
variableToPredict = "Age"
numberOfSamples = 2000
treeMaxDepth = 4

BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeData(treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth)


  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.6925


#### ASV + Shapley

In [15]:
asvCalc = ASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature)

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

showASVandShapleyFor(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc, progress=True)

ASV values for the first instance:
Progress of classes processed: 0.0%
Progress of classes processed: 0.5%
Progress of classes processed: 1.0%
Progress of classes processed: 1.5%
Progress of classes processed: 2.0%
Progress of classes processed: 2.5%
Progress of classes processed: 3.0%
Progress of classes processed: 3.5%
Progress of classes processed: 4.0%
Progress of classes processed: 4.5%
Progress of classes processed: 5.0%
Progress of classes processed: 5.5%
Progress of classes processed: 6.0%
Progress of classes processed: 6.5%
Progress of classes processed: 7.0%
Progress of classes processed: 7.5%
Progress of classes processed: 8.0%
Progress of classes processed: 8.5%


### Mean Prediction

In [9]:
# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(childNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)

In [33]:
bn.showMeanPredictionOfModel(variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

Mean prediction of model for the variable Age
Mean prediction value for the model: 0.577864520546124
Estimated value for shap explainer: 0.48624999999999996
Mean prediction if we only use the bayesian network: 0.5305312260310001


In [ ]:
dtAsNetwork = bn.obtainNetworkXTreeStructure(dtTreeClassifier, dataNoPrediction.columns)
bn.drawDecisionTree(dtAsNetwork)

In [35]:
from importlib import reload 
import asvFormula.bayesianNetworks.bayesianNetwork as bn

reload(asvFormula)
bn = reload(asvFormula.bayesianNetworks.bayesianNetwork)